# Project 6- Pipeline 4 - Predict

### Mine: Query Wikipedia by page id for content

In [1]:
import lib.wiki_module as wiki
import lib.encoding_module as enc
import lib.database_module as db
from sklearn.externals import joblib
model = joblib.load('../../../data/vectorizer.pkl')
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [38]:
page_id = '4143342'

In [39]:
page_text = wiki.query_page(page_id)

In [40]:
print page_text['text']

Marshyhope Creek is a 37.0-mile-long (59.5 km) tributary of the Nanticoke River on the Delmarva Peninsula. It rises in Kent County, Delaware, and runs through Caroline County, Maryland, and Dorchester County, Maryland. Marshyhope Creek begins in southwestern Kent County, Delaware, near the city of Harrington, and ends at the Nanticoke River, near Sharptown, Maryland. The entire watershed is in the coastal plain and reaches sea level at Brookview, Maryland. Federalsburg is the head of navigation. There are several small creeks on the western shore, including Faulkner Branch, Sullivan Branch, Capital Branch, Green Branch, and Horsepen Arm. On the eastern shore the small creeks include Tanyard Branch, Houston Branch, Jones Mill Branch, Double Fork Branch, Quarter Branch, Tomahawk Branch, and Prospect Point Branch. See alsoEdit List of rivers of Delaware ReferencesEdit United States Geological Survey. 7.5 minute series topographic quadrangles


### Parse: Encode page as document vector

In [41]:
search_term_vector = enc.get_searchterm_vector(model,page_text['text'])

In [42]:
search_vector =  search_term_vector[search_term_vector.keys()[0]][0]

In [43]:
cat_vectors = db.select_all_category_vectors()  #local_group_solution


Connected to server joshuacook.me.


In [44]:
category_ids, category_vectors = zip(*cat_vectors)

In [45]:
cos = cosine_similarity(category_vectors,search_vector.reshape(1,-1))
#print zip(category_ids, cos)

In [46]:
cos.argmax()
match1 = category_ids[cos.argmax()]
confidence1 = 2*cos.item(cos.argmax())
cos2 = np.delete(cos, cos.argmax())
category_ids2 = np.delete(category_ids,cos.argmax())
match2 = category_ids2[cos2.argmax()]
confidence2 = 2*cos2.item(cos2.argmax())

In [49]:
sql = """select category_name 
         FROM category
         WHERE category_id
         IN ('{}');
    """.format (match1)
match_name1 = db.execute_sql_statement(sql)
match_name1 = match_name1[0]
sql = """select category_name 
         FROM category
         WHERE category_id
         IN ('{}');
    """.format (match2)
match_name2 = db.execute_sql_statement(sql)
match_name2 = match_name2[0]
print "\n\nThe best fit category is {} with a {:.0f}% confidence.".format(match_name1[0],confidence1*100)
print "The next best fit is {} with a {:.0f}% confidence.".format(match_name2[0],confidence2*100)

Connected to server joshuacook.me.
Connected to server joshuacook.me.


The best fit category is Physical_quantities with a 31% confidence.
The next best fit is health care with a 26% confidence.
